# Subindo a STG para o banco

In [4]:
# Importando as bibliotecas 

import pandas as pd 
import sys
import os
from sqlalchemy import text
from sqlalchemy import create_engine
from dotenv import load_dotenv
from acessobanco import acessobanco
# from helpers import truncate_table



In [6]:
# Carrega variáveis de ambiente de arquivo específico para conexão com o banco
load_dotenv(dotenv_path="/home/danielpedro/Python/Projetos/SuperStorePY/Bancos/acessobanco.env")


True

In [7]:
# Cria engine de conexão com o banco PostgreSQL
# Esta engine será usada para todas as operações de banco no notebook
engine = acessobanco()

In [9]:
# Criando o dataframe

df = pd.read_excel('/home/danielpedro/Python/Projetos/SuperStorePY/Dataset/Sample - Superstore.xlsx', sheet_name='Orders')

In [10]:
# Visualizando os o top 5

df.head()

,Id,Id_Pedido,Data_Pedido,Data_Envio,Modo_Envio,Id_Cliente,Nome_Cliente,Segmento,Pais,Cidade,...,CE,Regiao,Id_Produto,Categoria,Sub_Categoria,Nome_Produto,Valor_Venda,Quantidade,Desconto,Lucro
0,646,CA-2017-126221,2017-12-30,2018-01-05,Standard Class,CC-12430,Chuck Clark,Home Office,United States,Columbus,...,47201,Central,OFF-AP-10002457,Office Supplies,Appliances,Eureka The Boss Plus 12-Amp Hard Box Upright V...,209.300,2,0.0,56.5110
1,907,CA-2017-143259,2017-12-30,2018-01-03,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,...,10009,East,FUR-BO-10003441,Furniture,Bookcases,"Bush Westfield Collection Bookcases, Fully Ass...",323.136,4,0.2,12.1176
2,908,CA-2017-143259,2017-12-30,2018-01-03,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,...,10009,East,TEC-PH-10004774,Technology,Phones,Gear Head AU3700S Headset,90.930,7,0.0,2.7279
3,909,CA-2017-143259,2017-12-30,2018-01-03,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,...,10009,East,OFF-BI-10003684,Office Supplies,Binders,Wilson Jones Legal Size Ring Binders,52.776,3,0.2,19.7910
4,1297,CA-2017-115427,2017-12-30,2018-01-03,Standard Class,EB-13975,Erica Bern,Corporate,United States,Fairfield,...,94533,West,OFF-BI-10002103,Office Supplies,Binders,"Cardinal Slant-D Ring Binder, Heavy Gauge Vinyl",13.904,2,0.2,4.5188


In [11]:
# Mapear colunas específicas se necessário

df = df.rename(columns={
    'Id': 'id',
    'Id_Pedido': 'id_pedido', 
    'Data_Pedido': 'data_pedido',
    'Data_Envio': 'data_envio',
    'Modo_Envio': 'modo_envio',
    'Id_Cliente': 'id_cliente',
    'Nome_Cliente': 'nome_cliente',
    'Segmento': 'segmento',
    'Pais': 'pais',
    'Cidade': 'cidade',
    'Estado': 'estado',
    'CE': 'ce',
    'Regiao': 'regiao',
    'Id_Produto': 'id_produto',
    'Categoria': 'categoria',
    'Sub_Categoria': 'sub_categoria',
    'Nome_Produto': 'nome_produto',
    'Valor_Venda': 'valor_venda',
    'Quantidade': 'quantidade',
    'Desconto': 'desconto',
    'Lucro': 'lucro'
})


In [16]:
# Gravando um os dados na tabela 

df.to_sql(
    name='orders',
    con=engine,
    schema='stg',
    if_exists='append',
    index=False,         # não envia o índice do DataFrame
    method="multi",      # insere em lote (batch)
    chunksize=5000       # 5.000 linhas por inserção
)

9994

In [15]:
# Realizando uma contagem dos dados usando o pandas

count = pd.read_sql("SELECT COUNT(*) as total_registros FROM stg.orders", engine).iloc[0,0]
print(f"Total de registros na tabela: {count}")

Total de registros na tabela: 0


In [14]:
# Truncate da tabela 

tabela = 'orders'
schema = 'stg' 

try:
    with engine.begin() as conn:
        print(f'Limpando tabela {schema}.{tabela}...')
        conn.execute(text(f'TRUNCATE TABLE {schema}.{tabela}'))
        print(f'Tabela {schema}.{tabela} truncada com sucesso!')
        
except Exception as e:
    print(f'Erro ao truncar a tabela {schema}.{tabela}: {e}')
    print('Detalhes do erro:', str(e))

Limpando tabela stg.orders...
Tabela stg.orders truncada com sucesso!
